In [1]:
from queue import Queue
import threading
from time import sleep
from first_UI.UI_init import start_pyqt
import program_2.factory_config as factory_config
from program_2.factory_config import in_stores
from program_2.factory_config import flow
from program_2.FactoryCalc import max_consumption
from program_2.FactoryCalc import pack_out_to_in
import neural_network_v2.learning as learning



# import neural_network.network as network
import program_1.modul as modul

from neural_network_v2 import network


calc_starting = True
itera = 0


def store_filling(stores, fill_value, indication="+"):
    if isinstance(stores, list):
        for i in range(len(stores)):
            if indication == "-":
                stores[i] = stores[i] - fill_value[i]
            else:
                stores[i] = stores[i] + fill_value[i]
    else:
        for item in stores:
            if item in fill_value:
                if indication == "-":
                    stores[item] = list(map(lambda x, y: x - y, stores[item], fill_value[item]))
                else:
                    stores[item] = list(map(lambda x, y: x + y, stores[item], fill_value[item]))


def initialized():   
    # запуск
    # инициализируем нейроны
    global neuron_matrix

    neural_config = network.neuron_scheme.copy()  # схема перцептрона хранится в модуле нейросети
    neural_config.insert(0, len(in_stores))  # входными нейронами будут in_stores
    neural_config.append(len(factory_config.flow))  # выходными нейронами будут потоки
    neuron_matrix = network.NeuronMatrix(neural_config)  # передаём получившуюся схему в конструктор нейрсети. Сохраняем ссылку на результат

    learning.neuron_matrix = neuron_matrix

# рассчитываем сколько в какой поток направила ресурсов нейросеть

# пропускаем запасы складов через функцию активации чтобы потом использовать как множитель потребления
temp_in_stores = {}
def resource_flow():
    excess = 0.2  # на сколько мы можем привышать потребление
    a = 10000  # чем больше значение, тем больше будет накапливаться буфера в складах (будет стремиться к этому значению)
    for item in in_stores:
        temp = in_stores[item][0] / a
        if temp > 1:
            temp = 1
        temp_in_stores[item] = [temp + excess]
        # изначально потребление будет стремиться к приходу ресурсов.
        # перемнажаем приход на полученые коофициенты запасов
        temp_in_stores[item] = [temp_in_stores[item][0] * factory_config.mine[item][0]]


# расчитываем сколько ресурса пришло в поток
def flow_calc():
    out_neural_data = neuron_matrix.get_data(len(neuron_matrix.all_layers)-1)
    # print (out_neural_data)
    for item in factory_config.flow_map:
        sum_out_neural = 0
        for i in factory_config.flow_map[item]:
            sum_out_neural += out_neural_data[i]
        for i in factory_config.flow_map[item]:
            flow[i] = (out_neural_data[i] / sum_out_neural) * temp_in_stores[item][0]


# запускаем цикл нейросети
def cycle_run():
    global flow
    count = 1
    for i in range(1):
        count += 1
        store_filling(in_stores, factory_config.mine)  # наполняем склады

        neuron_matrix.data_upd (modul.in_data_to_neur(modul.item_to_neur(in_stores)), 0) # записываем во входные нейроны значения скаладов пропущеные через нормализатор
        for j in range(len(neuron_matrix.all_layers) - 1):  # проходимся по сети функцией прямого распространения.
            # На выходе получаем значения потоков 0,0..1,0
            neuron_matrix.forWards(j)
        
        # рассчитываем сколько в какой поток направила ресурсов нейросеть 
        resource_flow()
        flow_calc()
        # запрашиваем калькуляцию у заводов. Данные уже у них в листе "flow"
        flow = max_consumption() # после этого на flow записывается сколько фактически забрали заводы, избыточные остаются. В дальнейшем избыток будет генерировать отрицательную ошибку
        # store_filling(in_stores, pack_out_to_in(), "-") # вычитаем из складов, то что взяли заводы/ больше не надо, заводы сами вычитают при калькуляции
        # генерируем ошибку заводов. Состоит из:
        # + ошибка складов out если выходов несколько пропорционально умножаем каждый
        factory_config.err_out_stor = modul.maximal_alignment(factory_config.out_stores)
        factory_config.err_out_stor = modul.midle_shift(factory_config.err_out_stor, 1) # получаем ошибку согласно заполненности

        # доп. средняя ошибка пропорции ресурсов. Если один ресурс = 1 другой 2, но второй из-за первого был использован лишь 1, то средняя = 1,5
        
        for fac in factory_config.all_factoty_list:
            temp_arr = []
            for i in fac.i_flow: # берём карту нейронов завода
                temp_arr.append(factory_config.flow_change[i]) # по ней по сдвигу вычисляем ошибку
            fac.err_request = modul.midle_shift(temp_arr, 1)

        # прим у нас есть flow (сколько было израсходовано), minimum (в max_consumption(), склько было сделано циклов), flow_change (то на сколько изменились от исходного расчёта)
        # + на каждый поток передаём ошибку складов + соответствующего ресурса

        for fac in factory_config.all_factoty_list:
            for i, err_req in zip(fac.i_flow, fac.err_request):
                factory_config.err_flow[i] = err_req + factory_config.err_out_stor[list(fac.produce.keys())[0]][0] #пока расчитываем на 1 продукцию

        # считаем ошибку in складов (чем больше запас, тем больше ошибка)
        temp_modle_shift = modul.midle_shift(modul.maximal_alignment(in_stores), -1)
        for item in factory_config.flow_map:
            temp_sum = 0
            for i in factory_config.flow_map[item]:
                temp_sum += flow[i]

            for i in factory_config.flow_map[item]:
                factory_config.temp_flow[i] = flow[i] / temp_sum * temp_modle_shift[item][0]
        # ошибка in складов записана в factory_config.temp_flow (массив нейронов)
        # финализируем ошибку
        # берём ошибку складов в виде уже последовательности неронов
        # берем ошибку фактори
        # теперь сливаем сумму на еррор слой
        errArray = neuron_matrix.get_data(len(neuron_matrix.all_layers)-1, True)
        for i in range(len(errArray)):
            errArray[i] = factory_config.temp_flow[i] + factory_config.err_flow[i]
        neuron_matrix.data_upd(errArray,len(neuron_matrix.all_layers)-1, True)

        for j in range(len(neuron_matrix.all_layers) - 1, 0, -1):  # проходимся по сети функцией обратного распространения
            neuron_matrix.backWards(j)
        #запускаем модуль обучения
        learning.learn_program()


        

# запускаем потоки
def thread_runner():
    initialized()

    q = Queue()
    print("инициализация UI")
    q.put("инициализация UI")
    thr_2 = threading.Thread(target=start_pyqt, args=("pyqt", q))
    thr_2.start()
    q.join()
    print("UI загружен")

    while thr_2.is_alive():
        sleep(2)
        if calc_starting:
            thr_1 = threading.Thread(target=cycle_run)
            thr_1.start()
            thr_1.join()     

def not_thread_runner():
    initialized()      

if __name__ == '__main__':
    not_thread_runner()

In [742]:
for i in range(1000):
    cycle_run()
    itera += 1
    factory_config.out_stores['reinf_iron_plate'][0] -= 2
    factory_config.out_stores['rotor'][0] -= 1

print("итерация - ", itera)

print("\nмайнинг")
print(factory_config.mine)

print("\nсклады сырья")
print(factory_config.in_stores)

print("\nсклады продукции")
print(factory_config.out_stores)

print("\nактивность фактори циклов")
for fac in factory_config.all_factoty_list:
    print(fac.name)
    print([factory_config.flow_map2[x] for x in fac.i_flow], " - " , fac.minimum)

итерация -  266000

майнинг
{'iron': [100], 'copper': [30], 'coal': [10]}

склады сырья
{'iron': [7900.697159934401], 'copper': [7974.790290180513], 'coal': [7990.80824080378]}

склады продукции
{'reinf_iron_plate': [1726326.1445502345], 'rotor': [1718271.4169317184]}

активность фактори циклов
f_reinf_iron_plate_1
['iron']  -  2.2668447482069296
f_reinf_iron_plate_2
['iron', 'copper']  -  5.4424336994288245
f_reinf_iron_plate_3
['iron']  -  2.0333021902448314
f_rotor_1
['iron']  -  0.4223587895503891
f_rotor_2
['iron', 'copper']  -  0.9363254358172532
f_rotor_3
['iron', 'copper', 'coal']  -  3.3336030166178783


In [2]:
factory_config.mine['iron'] = [100]
factory_config.mine['copper'] = [30]
factory_config.mine['coal'] = [10]


In [4]:
# финализируем ошибку
# берём ошибку складов в виде уже последовательности неронов
# берем ошибку фактори
# теперь сливаем сумму на еррор слой
errArray = neuron_matrix.get_data(len(neuron_matrix.all_layers)-1, True)
for i in range(len(errArray)):
    errArray[i] = factory_config.temp_flow[i] + factory_config.err_flow[i]
print(errArray)
neuron_matrix.data_upd(errArray,len(neuron_matrix.all_layers)-1, True)
print(neuron_matrix.get_data(len(neuron_matrix.all_layers)-1, True))


[-0.026164689769529348, -0.02592358390928607, 0.014519638054127272, -0.02827981622227002, -0.02223690103695204, -0.028347982376432452, 0.011011608757433853, -0.026395125188330378, 0.022197815901871008, 0.10961903578936816]

[-0.02109366734633067, 0.0246307712813296, -0.06681810597399093, -0.02109366734633067, 0.02109366734633067, 0.07220446640204266, -0.03001713170938132, 0.15274474465461996, 0.090691398142553, -0.18015514075818095]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[-0.04725835711586002, -0.001292812627956469, -0.05229846791986366, -0.04937348356860069, -0.0011432336906213696, 0.04385648402561021, -0.01900552295194747, 0.1263496194662896, 0.11288921404442401, -0.0705361049688128]
ошибок слой
ошибок слой
ошибок слой
ошибок слой
ошибок слой
ошибок слой
ошибок слой
ошибок слой
ошибок слой
ошибок слой
[-0.04725835711586002, -0.001292812627956469, -0.05229846791986366, -0.04937348356860069, -0.0011432336906213696, 0.04385648402561021, -0.01900552295194747, 0.1263496194662896, 0.1128892140444

In [3]:
factory_config.err_out_stor

{'reinf_iron_plate': [0.018198158919985108], 'rotor': [-0.018198158919985108]}

In [27]:
for fac in factory_config.all_factoty_list:
    print(fac)
    request_coef = [x for x in fac.request_coef.values()]
    print (request_coef, "---", flow)
    max_cicle = list(map(lambda i_flow, coef : flow[i_flow] / coef, fac.i_flow, request_coef))
    print (max_cicle)
    minimum = min(max_cicle)

    for i in range(len(fac.i_flow)):
        print(flow_change[fac.i_flow[i]])

[1.0] --- [0.09908837019907302, 0.09635189664367756, 0.06417036316468926, 0.09661571372771487, 0.09616855532794714, 0.08376232450988014, 0.03868929538562592, 0.09711326634689839, 0.09711326634689839, 0.09711326634689839]
[0.09908837019907302]
0.0
[1.0, 0.666] --- [0.09908837019907302, 0.09635189664367756, 0.06417036316468926, 0.09661571372771487, 0.09616855532794714, 0.08376232450988014, 0.03868929538562592, 0.09711326634689839, 0.09711326634689839, 0.09711326634689839]
[0.09635189664367756, 0.09635189664367756]
0.0
-0.04992528389775999
[1.0] --- [0.09908837019907302, 0.09635189664367756, 0.06417036316468926, 0.09661571372771487, 0.09616855532794714, 0.08376232450988014, 0.03868929538562592, 0.09711326634689839, 0.09711326634689839, 0.09711326634689839]
[0.09661571372771487]
0.0
[1.0] --- [0.09908837019907302, 0.09635189664367756, 0.06417036316468926, 0.09661571372771487, 0.09616855532794714, 0.08376232450988014, 0.03868929538562592, 0.09711326634689839, 0.09711326634689839, 0.09711326